In [ ]:
class TSP:
    
    ## Initialization ##
    def __init__(self, cities):
        self.cities = cities
        self.n = len(self.cities)
        self.dist = [[0] * self.n for i in range(self.n)]
        pass
    
    
    ## Calculate the distance between two cities #
    def twoCitiesDistance(self, city1, city2):
        return sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1]) ** 2)
        pass
    
    
    ## Generate a distance matrix ##
    def distanceMatrix(self):
        for i in range(self.n):
            for j in range(i, self.n):
                self.dist[i][j] = self.dist[j][i] = self.twoCitiesDistance(self.cities[i], self.cities[j])
        pass
    
    
    ## Calculate the total distance of a tour ##
    def totalDistance(self, tour):
        start = tour[:self.n-1]
        end = tour[1:]
        distance = self.dist[tour[0]][tour[self.n-1]]
        for i, j in zip(start,end):
            distance += self.dist[i][j]   
        return distance
        pass
    
    
    ## Judge whether two edges (p1<-->p2, p3<-->p4) are overlapped ##
    # Input: city index--p1, p2, p3, p4
    # Output: True of False
    def isOverlap(self, p1, p2, p3, p4):
 
        x1 = self.cities[p1][0]
        y1 = self.cities[p1][1]
        x2 = self.cities[p2][0]
        y2 = self.cities[p2][1]
        x3 = self.cities[p3][0]
        y3 = self.cities[p3][1]
        x4 = self.cities[p4][0]
        y4 = self.cities[p4][1]
        
        temp = (x1-x2)*(y3-y4) - (y1-y2)*(x3-x4)
        if temp != 0:
            # coordinate of the intersection point (x,y)
            x = ((x1*y2-y1*x2) * (x3-x4) - (x1-x2) * (x3*y4-y3*x4)) / temp
            y = ((x1*y2-y1*x2) * (y3-y4) - (y1-y2) * (x3*y4-y3*x4)) / temp
            # check if intersect. point is on two edges
            if min(x1,x2)<x<max(x1,x2) and min(y1,y2)<y<max(y1,y2) and min(x3,x4)<x<max(x3,x4) and min(y3,y4)<y<max(y3,y4):
                return True
        return False
        pass
    
    
    ## Switch overlapped edges ##
    def processOverlap(self, tour):
        
        isOverlap_flag = True
        # new_tour = [idx start, idx 1, ..., idx n-1, idx start]
        new_tour = copy.deepcopy(tour)
        new_tour.append(tour[0])
        counter = 0
        
        while isOverlap_flag == True:
            counter += 1
            if counter > 5000:
                break
            
            isOverlap_flag = False
    
            for i in range(self.n-2):
                for j in range(i+2, self.n):
                    # edge 1: p1<-->p2; edge 2: p3<-->p4
                    p1, p2, p3, p4 = new_tour[i], new_tour[i+1], new_tour[j], new_tour[j+1]
                    
                    # Edges 1 and 2 are overlapped: switch them if new distance < old distance. 
                    # After switching: p1<-->p3; p2<-->p4
                    if self.isOverlap(p1, p2, p3, p4):
                        old_dist = self.dist[p1][p2] + self.dist[p3][p4]
                        new_dist = self.dist[p1][p3] + self.dist[p2][p4]
                        if new_dist < old_dist:
                            new_tour[i+1], new_tour[j] = new_tour[j], new_tour[i+1]
                            new_tour[i+2:j] = reversed(new_tour[i+2:j])
                            isOverlap_flag = True
                            break
                                       
        return new_tour
        pass
    
    
    ## Greedy algorithm ##
    def greedy(self):

        best_tour = []
        shortest_dist = float('inf')
        
        current_city = randint(0, self.n-1)
        unvisited_cities = set(list(range(0,current_city)) + list(range(current_city+1,self.n)))
        tour = [current_city]

        while unvisited_cities:
            next_city = min(unvisited_cities, key=lambda city: self.dist[current_city][city])
            unvisited_cities.remove(next_city)
            tour.append(next_city)
            current_city = next_city
                
        return tour
        pass
    
    
    ## SA algorithm ##
    def sa(self, tour):
        # parameters
        iter_1 = 5     
        iter_2 = 30000
        T0 = 1.2
        Tf = 1                           
        alpha = 0.99                  
        
        # initialize a tour by greedy algorithm
        tour = self.greedy() 
        
        # calculate distance of the initialized tour
        shortest_dist = 0         
        for j in range(self.n-1):
            shortest_dist = shortest_dist + self.dist[tour[j]][tour[j + 1]]
        shortest_dist = shortest_dist + self.dist[tour[-1]][tour[0]]
        
        best_tour = tour.copy()
        dist_now = shortest_dist
        tour_now = best_tour.copy()

        for i in range(iter_1):
            for k in range(iter_2):
                
                # generate a new tour
                tour_new = [0 for i in range(self.n)]
                n1, n2 = randint(0, self.n-1), randint(0, self.n-1)
                n = [n1,n2]
                n.sort()
                n1, n2 = n
                
                if n1 > 0:
                    tour_new[0:n1] = tour_now[0:n1]
                    tour_new[n1:n2+1] = tour_now[n2:n1-1:-1]
                    tour_new[n2+1:self.n] = tour_now[n2+1:self.n]
                else:
                    tour_new[0:n1] = tour_now[0:n1]
                    tour_new[n1:n2+1] = tour_now[n2::-1]
                    tour_new[n2+1:self.n] = tour_now[n2+1:self.n]
                 
                # calculate the distance of the new tour
                distance = 0
                for j in range(self.n - 1):
                    distance = distance + self.dist[tour_new[j]][tour_new[j+1]]
                distance = distance + self.dist[tour_new[-1]][tour_new[0]]
                
                # judge if the new tour is a better tour
                if distance <= dist_now:
                    dist_now = distance
                    tour_now = tour_new.copy()
                if distance > dist_now:
                    deltaf = distance - dist_now
                    if random() < exp(-deltaf/T0):
                        dist_now = distance
                        tour_now = tour_new.copy()
                if distance < shortest_dist:
                    shortest_dist = distance
                    best_tour = tour_new.copy()
            # update temparature
            T0 = alpha * T0    
            
        return best_tour
        pass
    
        
    ## main function of solving ##
    def solve(self):
        
        # generate distance matrix
        self.distanceMatrix()
        # intialize a tour by breedy
        initial_tour = self.greedy()     
        # improve the initial tour by sa algorithm
        best_tour = self.sa(initial_tour) 
        # un-overlap the edges of tour
        best_tour = self.processOverlap(best_tour)
        
        return best_tour, self.totalDistance(best_tour)
        pass

In [2]:
if __name__ == '__main__':
    
    import sys
    import copy
    from random import *
    from math import *
    import numpy as np
    from common import print_tour, read_input

In [3]:
for i in range(4,8):
    
    file_name = 'input_'+str(i)+'.csv'
    input_data = read_input(file_name)
    solution = TSP(input_data)
    best_tour, shortest_dist = solution.solve()
    
    #print('===== Best Tour for', file_name, '=====')
    #print_tour(best_tour)
    print('===== Shortest Distance for', file_name, '=====\n', shortest_dist)

===== Shortest Distance for input_4.csv =====
 11311.508371536866
===== Shortest Distance for input_5.csv =====
 21826.813155649994
===== Shortest Distance for input_6.csv =====
 45661.40860520704
===== Shortest Distance for input_7.csv =====
 90448.59145517892
